In [1]:
import sys
import geomagio
from obspy.core import UTCDateTime
import urllib.request, json
import numpy as np
import matplotlib.pyplot as plt
import geomagio
import datetime

%matplotlib inline
endtime =  UTCDateTime(datetime.datetime.utcnow())-60*86400.
starttime =  endtime-60*86400.
print(starttime, endtime)

#url = "https://geomag.usgs.gov/ws/edge/?id=BOU&format=json"
#url = url + "&starttime=2017-07-10T00:00:00Z"
#url = url + "&endtime=2017-07-18T00:00:00Z"
#url = url + "&type=adjusted"
#response = urllib.request.urlopen(url)
#data = json.loads(response.read())

2017-03-28T23:35:35.792843Z 2017-05-27T23:35:35.792843Z


In [2]:
OBSERVATORIES = ('BOU', 'BRW', 'BSL', 'CMO', 'DED', 'FRD',
                 'FRN', 'GUA', 'HON', 'NEW', 'SHU', 'SIT', 
                 'SJG', 'TUC')

CHANNELS = ('X', 'Y', 'Z')

nchannels = len(CHANNELS)*len(OBSERVATORIES)

input_factory = geomagio.edge.EdgeFactory()
#starttime = datetime.datetime.utcnow()
#endtime = starttime

for i, obs in enumerate(OBSERVATORIES):
    timeseries = input_factory.get_timeseries(
        observatory = obs,
        channels = CHANNELS,
        type = 'variation',
        interval = 'minute',
        starttime = starttime,
        endtime = endtime)
    if i == 0:
        data = np.zeros([len(CHANNELS)*len(OBSERVATORIES),timeseries[0].data.shape[0]],dtype='float')
    print(i,obs,timeseries[0].data.shape, np.nanmean(timeseries[0].data),
          np.nanmean(timeseries[1].data),np.nanmean(timeseries[2].data))
    
    for j in range(len(CHANNELS)):
        length = timeseries[j].data.shape[0]
        data[i*len(CHANNELS)+j,0:length] = (timeseries[j].data[0:length])

0 BOU (86401,) 20565.2621764 3340.49741308 47192.7953605
1 BRW (86401,) 8495.92124494 3350.41406633 56827.0508671
2 BSL (86401,) 23265.8480219 316.42638562 40910.4215459
3 CMO (86401,) 11340.6979819 4885.04679147 55328.9556046
4 DED (86401,) 8626.45338325 2875.22884565 56734.225341
5 FRD (86401,) 21243.8039132 -3523.9905709 45668.8194582
6 FRN (86401,) 22692.6491737 5804.62253477 42120.4937506
7 GUA (86401,) 35236.10182 862.302829643 7668.00561145
8 HON (86401,) 26889.4595616 5046.48601805 21072.6571014
9 NEW (86401,) 17079.049608 5504.79625427 50727.9198405
10 SHU (86401,) 18903.3529718 6016.27063723 48634.1288748
11 SIT (86401,) 14467.869073 6308.67291106 52451.4425862
12 SJG (86401,) 26354.815969 -5411.51430346 24982.9363562
13 TUC (86401,) 24082.730606 4582.42401287 40219.9390275


In [3]:
import pickle
DIR = "/data/st/geomag_2010_2016_xyzf/"
#FILENAME = 'F_2015_minutes.pkl'
#timeseries=np.array(pickle.load(open(DIR + FILENAME,'rb'),  encoding="latin1"))
#for o in range(timeseries.shape[0]):
#    timeseries2[o*4+0,:] = timeseries[o,:]
FILENAME = 'X_2015_minutes.pkl'
timeseries=np.array(pickle.load(open(DIR + FILENAME,'rb'),  encoding="latin1"))
timeseries2=np.zeros([len(OBSERVATORIES)*len(CHANNELS),timeseries.shape[1]],dtype='float')
for o in range(timeseries.shape[0]):
    timeseries2[o*3+0,:] = timeseries[o,:]
FILENAME = 'Y_2015_minutes.pkl'
timeseries=np.array(pickle.load(open(DIR + FILENAME,'rb'),  encoding="latin1"))
for o in range(timeseries.shape[0]):
    timeseries2[o*3+1,:] = timeseries[o,:]
FILENAME = 'Z_2015_minutes.pkl'
timeseries=np.array(pickle.load(open(DIR + FILENAME,'rb'),  encoding="latin1"))
for o in range(timeseries.shape[0]):
    timeseries2[o*3+2,:] = timeseries[o,:]
timeseries = timeseries2
    
print("timeseries.shape=",timeseries.shape)

data = timeseries[:,:]
nchannels = data.shape[0]
print("nchannels=",nchannels)

timeseries.shape= (42, 525600)
nchannels= 42


In [4]:
import sting

lags = np.array(range(1))
batch_size=1

# Average
window = 60
sampled_values, time = sting.raw_to_sampled(data, window)
print("Done raw_to_sampled, sampled_values.shape", sampled_values.shape)

scalers_file = 'scalers_window{0:03d}_lags{1:03d}.pkl'.format(window, np.max(lags))
scalers = pickle.load(open(scalers_file,'rb'),  encoding="latin1")

train_scaled, test_scaled, scalers2 = sting.sampled_to_scaled(sampled_values, 
                                                              time, nchannels, 
                                                              lags, batch_size, 
                                                              scalers=scalers)
print(train_scaled.shape,test_scaled.shape)


time.shape (8760,)
nchannels= 42


TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
train_scaled.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import model_from_json

json_file = "model_window{0:03d}_lags{1:03d}.json".format(window, np.max(lags))
model_file= "lstm_model_window{0:03d}_lags{1:03d}.h5".format(window, np.max(lags))

json_file = open(json_file, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(model_file)
loaded_model.reset_states()

In [ ]:
repeats = 1
predictions = np.zeros([test_scaled.shape[0],test_scaled.shape[1]])
predictions[0,:] = test_scaled[0,:]

for r in range(repeats):    
    for i in (range(test_scaled.shape[0]-2)):
        # make one-step forecast
        if i > int(0.7*test_scaled.shape[0]):
            X = yhat
            yhat = sting.forecast_lstm(loaded_model, 1, X)
            yhat = yhat[0,:]
            predictions[i+1,:] = yhat
        else:
            X = test_scaled[i, :]
            yhat = sting.forecast_lstm(loaded_model, 1, X)
            yhat = yhat[0,:]
            predictions[i+1,:] = test_scaled[i+1, :]


In [ ]:
predictions = predictions.reshape([predictions.shape[0],lags.shape[0],nchannels])
print(predictions.shape)
print(test_scaled.shape)

In [ ]:
inverted_predictions = predictions.reshape([predictions.shape[0],lags.shape[0],nchannels])
inverted_test = test_scaled.reshape([predictions.shape[0],lags.shape[0],nchannels])

for c in range(len(scalers2)):
    sc = scalers2[c]
    #inverted_predictions[:,:,c] = 
    #inverted_predictions[:,:,c] = sc.inverse_transform(inverted_predictions[:,:,c])
    #inverted_test[:,:,c] = 
    #inverted_test[:,:,c] = sc.inverse_transform(inverted_test[:,:,c])

In [ ]:
#plt.plot(inverted_predictions[:,len(lags)-1,12])
#plt.plot(inverted_test[:,len(lags)-1,12],'--')

from scipy.stats import pearsonr

for c in range(nchannels):
    plt.figure(c)
    obs = int(np.floor(c/len(CHANNELS)))
    comp = c % len(CHANNELS)
    x = inverted_test[:,len(lags)-1,c]
    y = inverted_predictions[:,len(lags)-1,c]
    plt.plot(x)
    plt.plot(y,'--')
    #pyplot.ylim(predictions[:,len(lags)-1,c].min(),predictions[:,len(lags)-1,c].max())
    plt.xlim(int(0.68*test_scaled.shape[0]),int(0.75*test_scaled.shape[0]))
    plt.ylim(np.array([x,y]).min(),np.array([x,y]).max())
    corr = pearsonr(x,y)
    plt.title("{0} ({1}), pearson r={2:.2f}, p={3:.3f}".format(OBSERVATORIES[obs],CHANNELS[comp],corr[0],corr[1]))
    plt.show()

In [ ]:
nchannels

In [ ]:
#timeseries = input_factory.get_timeseries(
#        observatory = obs,
#        channels = CHANNELS,
#        type = 'variation',
#        interval = 'minute',
#        starttime = UTCDateTime('2013-03-28T20:41:16.951677Z'),
#        endtime = UTCDateTime('2014-03-28T20:41:16.951677Z'))